In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

# LOCATION DATASET PREPARATION

## 278 STATIONS

###DATA PREPARATION

The available data on the localization of the 278 weather stations is provided with the UTM system. As is quite complex, I convert the values to the usual latitude longitude system in degree decimals.

In [ ]:
%pip install utm

In [ ]:
pip install pyproj

In [ ]:
from google.colab import files
up = files.upload()
import io
loc  = pd.read_csv(io.BytesIO(up['locations.tsv']), sep='\t')

loc.head(10)

In [ ]:
loc = loc[['idStazione', 'Stazione', 'Provincia','UTM_Nord', 'UTM_EST','latitude', 'longitude']]

In [ ]:
loc['UTM_Nord'] = loc['UTM_Nord'].astype(float)
loc['UTM_EST'] = loc['UTM_EST'].astype(float)


In [ ]:
import utm
def converter(v1, v2):
# Define the UTM coordinates
  utm_easting = v2  # Example UTM easting
  utm_northing = v1   # Example UTM northing
  utm_zone_number = 32  # Example UTM zone number (North Italy)
  utm_zone_letter = 'T'  # Example UTM zone letter (North Italy)


  #convert from UTM to latitude and longitude measured in decimal of degree
  lat, lon = utm.to_latlon(utm_easting, utm_northing, utm_zone_number, utm_zone_letter)
  return lat, lon

# Apply the converter function to create 'latitude' and 'longitude' columns
loc[['latitude', 'longitude']] = loc[['UTM_Nord', 'UTM_EST']].apply(lambda row: pd.Series(converter(row['UTM_Nord'], row['UTM_EST'])), axis=1)

### DATA VISUALIZATION
I create a map that shows the distibution of the 278 weather stations around Lombardy (Italy)

In [ ]:
import folium  #278

# create a map with the 278 stations
m = folium.Map(location=[loc.at[1, "latitude"],loc.at[1, "longitude"]], zoom_start=12)

# add markers to the map
for i in range(1,len(loc)):
  try:
    folium.Marker(location=[loc.at[i, "latitude"],loc.at[i, "longitude"]], popup=loc.at[i, "0"]).add_to(m)
  except:
    print(i)

In [ ]:
m

## 160 STATIONS

### DATA PREPARATIONS

In [ ]:
low_l =  [i.lower() for i in l]

In [ ]:
filtered_pos = pos[pos['Stazione'].isin(low_l)]

In [ ]:
from google.colab import files
filtered_pos.to_csv('location_282_381_160.csv', index=None, header=True)
files.download('location_282_381_160.csv')

### DATA VISUALIZATION
I create a map that shows the distibution of the 160 weather stations chosen for the MES estimation around Lombardy (Italy)

In [ ]:
import folium  #160

# create a map with the 160 stations
m = folium.Map(location=[filtered_pos.at[1, "latitude"],filtered_pos.at[1, "longitude"]], zoom_start=12)

# add markers to the map
for i in range(1,len(filtered_pos)):
  try:
    folium.Marker(location=[filtered_pos.at[i, "latitude"],filtered_pos.at[i, "longitude"]], popup=filtered_pos.at[i, "0"]).add_to(m)
  except:
    print(i)

In [ ]:
m

# DATA PREPARATION AND CLEANING - 29 NEW STATIONS
For testing the goodness of fit of the prediction model through kriging I look for a new sample of stations for a period of 100 months so to use all the 160 stations as training sample.

## MONTHLY RAINFALL DATA PREPARATION

In [ ]:
rem_monthly_df = monthly_df.drop(columns =lim_l)

In [ ]:
find_complete_sequence(rem_monthly_df.iloc[:,1:], 100)

In [ ]:
l2 = [ 'Agrate Brianza v.Ferrario',
'Asola Sorbara',
'Bagolino Valle Dorizzo',
'Barni',
'Bienno',
'Bormio v.Monte Braulio',
'Busto Arsizio v.Magenta',
'Carate Brianza v.De Gasperi',
'Castiglione delle Stiviere',
"Chiesa in Valmalenco Alpe dell' Oro",
'Clusone Campo Sportivo',
'Crema v.Camporelle',
'Crema v.XI febbraio',
'Darfo Boario Terme',
'Darfo Boario Terme SS 42',
'Gera Lario Fuentes',
'Lomello SS 211',
'Lonate Pozzolo v.Veneto',
'Mariano Comense v.le Lombardia',
'Molteno Cimitero',
'Morbegno v.Cortivacci',
'Nibionno v.della Chiesa',
'San Pellegrino Terme P.za Alighieri',
'Sermide e Felonica SP 91',
'Somma Lombardo v.Facchinetti',
'Sondrio v.Mazzini',
'Tavazzano con Villavesco Enel suolo',
'Varese v.Vidoletti',
'Varzi v.Mazzini'
]

In [ ]:
rem_monthly_df = rem_monthly_df[l2]
rem_monthly_df = rem_monthly_df.iloc[154:254,1:]

In [ ]:
from google.colab import files
rem_monthly_df.to_csv('new_stations_29.csv', index=None, header=True)
files.download('new_stations_29.csv')

## COVARIANCE DATA PREPARATION

### HUMIDITY, TEMPERATURE, SOLAR RADIATION, WIND SPEED

In [ ]:
from google.colab import files
up = files.upload()
import io
df_u2  = pd.read_csv(io.BytesIO(up['COOV29 - UMIDITA.csv'])) #humidity

In [ ]:
from google.colab import files
up = files.upload()
import io
df_t2  = pd.read_csv(io.BytesIO(up['COOV29 - TEMPERATURA.csv'])) #temperature

In [ ]:
from google.colab import files
up = files.upload()
import io
df_r2  = pd.read_csv(io.BytesIO(up['COOV29 - RADIAZIONE.csv']))  #solar radiation

In [ ]:
from google.colab import files
up = files.upload()
import io
df_v2  = pd.read_csv(io.BytesIO(up['COOV29 - VENTO.csv']))  #wind speed

In [ ]:
df_u2 = df_u2[l2]  #humidty
df_t2 = df_t2[l2]  #temperature
df_r2 = df_r2[l2]  #solar radiation
df_v2 = df_v2[l2]  #wind speed

In [ ]:
modify_dataframe(df_u2)
modify_dataframe(df_t2)
modify_dataframe(df_r2)
modify_dataframe(df_v2)

In [ ]:
df_u2 = df_u2.replace(-999, np.nan)
df_t2 = df_t2.replace(-999, np.nan)
df_r2 = df_r2.replace(-999, np.nan)
df_v2 = df_v2.replace(-999, np.nan)

Monthly Aggregation

In [ ]:
monthly_df_u2 = create_monthly_df_u2(df_u2)
monthly_df_t2 = create_monthly_df_t2(df_t2)
monthly_df_r2 = create_monthly_df_r2(df_r2)
monthly_df_v2 = create_monthly_df_v2(df_v2)


In [ ]:
monthly_df_u2 =  monthly_df_u2/10
monthly_df_t2 = monthly_df_t2/10
monthly_df_r2 = monthly_df_r2/10
monthly_df_v2 = monthly_df_v2/10

In [ ]:
monthly_df_u2 = monthly_df_u2.reset_index()
monthly_df_t2 = monthly_df_t2.reset_index()
monthly_df_r2 = monthly_df_r2.reset_index()
monthly_df_v2 = monthly_df_v2.reset_index()

In [ ]:
monthly_df_u2 = monthly_df_u2.iloc[154:254,:]
monthly_df_t2 = monthly_df_t2.iloc[154:254,:]
monthly_df_r2 = monthly_df_r2.iloc[154:254,:]
monthly_df_v2 = monthly_df_v2.iloc[154:254,:]

In [ ]:
from google.colab import files
monthly_df_u2.to_csv('new_stations_29_umid.csv', index=None, header=True)
files.download('new_stations_29_umid.csv')

In [ ]:
from google.colab import files
monthly_df_t2.to_csv('new_stations_29_temp.csv', index=None, header=True)
files.download('new_stations_29_rad.csv')

In [ ]:
from google.colab import files
monthly_df_r2.to_csv('new_stations_29_rad.csv', index=None, header=True)
files.download('new_stations_29_rad.csv')

In [ ]:
from google.colab import files
monthly_df_v2.to_csv('new_stations_29_vento.csv', index=None, header=True)
files.download('new_stations_29_vento.csv')

### SEASONS

In [ ]:
pattern = [2, 3, 3, 3, 4, 4, 4, 1, 1, 1, 2,2] #3 autunno

# Calculate the length of the pattern
pattern_length = len(pattern)

start_index = 0

rem_monthly_df['Season'].iloc[start_index:] = [pattern[i % pattern_length] for i in range(start_index, len(rem_monthly_df))]

In [ ]:
rem_monthly_df['Autumn_Spring'] = rem_monthly_df['Season'].apply(Autumn_Spring)

In [ ]:
rem_monthly_df['Autumn_Winter'] = rem_monthly_df['Season'].apply(Autumn_Winter)

In [ ]:
rem_monthly_df['Autumn_Summer'] = rem_monthly_df['Season'].apply(Autumn_Summer)

In [ ]:
rem_monthly_df['Autumn'] = rem_monthly_df['Season'].apply(Autumn)

In [ ]:
from google.colab import files
rem_monthly_df.to_csv('new_stations_29_seasons.csv', index=None, header=True)
files.download('new_stations_29_seasons.csv')

## LOCATION DATA OF THE STATIONS PREPARATION

In [ ]:
low_l2 =  [i.lower() for i in l2]

In [ ]:
filtered_pos2 = pos[pos['Stazione'].isin(low_l2)]

In [ ]:
import folium  #29

# create a map with the 29 stations
m = folium.Map(location=[filtered_pos2.at[1, "latitude"],filtered_pos2.at[1, "longitude"]], zoom_start=12)

# add markers to the map
for i in range(1,len(filtered_pos2)):
  try:
    folium.Marker(location=[filtered_pos2.at[i, "latitude"],filtered_pos2.at[i, "longitude"]], popup=filtered_pos2.at[i, "0"]).add_to(m)
  except:
    print(i)

In [ ]:
from google.colab import files
filtered_pos2.to_csv('loc_new_stations_29.csv', index=None, header=True)
files.download('loc_new_stations_29.csv')

## DISTANCE DATA PREPARATION

In [ ]:
data = np.empty((28, 29))
data[:] = np.nan

dist2 = pd.DataFrame(data)

In [ ]:
for i in range(0,29):
  z = 0
  for j in range(0,29):
    if i != j:
      loc1 = (df.at[i,"latitude"],df.at[i,"longitude"])
      loc2 = (df.at[j,"latitude"],df.at[j,"longitude"])
      dist2.at[z,i] = hs.haversine(loc1,loc2,unit=Unit.KILOMETERS)
      z += 1


In [ ]:
from google.colab import files
dist2.to_csv('dist_29.csv', index=None, header=True)
files.download('dist_29.csv')

# PREDICTION OF KRIGING FOR SQUARE AREA AROUND LOMBARDY

## DATA PREPARATION


In [ ]:
min(loc["latitude"]), max(loc["latitude"]), min(loc["longitude"]), max(loc["longitude"])

(44.6837785, 46.49442681, 8.573921779, 10.88713683)

In [ ]:
lat_range = np.arange(44.68, 46.49, 0.1)
long_range = np.arange(8.57, 10.88, 0.1)

# Create lists for latitudes and longitudes
lat = []
long = []

# Populate the lists using nested loops
for i in lat_range:
    for j in long_range:
        lat.append(i)
        long.append(j)

In [ ]:
from google.colab import files
up = files.upload()
import io
pred  = pd.read_csv(io.BytesIO(up['kriging1.csv']))

In [ ]:
import folium

# Assuming you have a DataFrame named 'df' with columns 'latitude', 'longitude', and '0'

# create a map centered on a specific location
m = folium.Map(location=[pred.at[1, "V1"], pred.at[0, "V1"]], zoom_start=12, tiles='Stamen Terrain')

# add markers to the map
for i in range(0,456):
    try:
        if pred.iloc[2, i]<12 :
            marker_color = 'green'  # Set the marker color based on your condition
        elif pred.iloc[2, i]<20:
            marker_color = 'orange'  # Set a different color if the condition is not met
        else:
            marker_color = 'red'

        # Create a custom icon for the marker with the specified color
        icon = folium.Icon(color=marker_color)

        folium.Marker(
            location=[pred.iloc[1, i], pred.iloc[0, i]],
            popup=i,
            icon=icon
        ).add_to(m)
    except:
        print(i)

# Display the map
m